In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import pyodbc
import warnings
from sklearn.neighbors import KNeighborsClassifier

pd.set_option('display.max_columns', None)

leafspy = pd.read_csv('leafspy.csv')
weather_yokohama = pd.read_csv('weather_yokohama.csv')
weather_ebina = pd.read_csv('weather_ebina.csv')
illuminance = pd.read_csv('illuminance.csv')
precipitation_yokohama = pd.read_csv('precipitation_yokohama.csv')
precipitation_ebina = pd.read_csv('precipitation_ebina.csv')
tennki = pd.read_csv('tennki.csv')

leafspy = leafspy.drop(['Unnamed: 0'], axis=1)
weather_yokohama = weather_yokohama.drop(['Unnamed: 0'], axis=1)
weather_ebina = weather_ebina.drop(['Unnamed: 0'], axis=1)
illuminance = illuminance.drop(['Unnamed: 0'], axis=1)

D:\Anaconda\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\ELMO_HX\AppData\Local\Temp\ipykernel_35380\2403253388.py:16: DtypeWarning: Columns (6,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  precipitation_ebina = pd.read_csv('precipitation_ebina.csv')


# 温度単位が摂氏度と華氏度で混在しているため、すべて摂氏度に変換する必要があります。

In [2]:
def FIX_AMBIENT(data):
    for i in range(1,len(data)):
        if data['AMBIENT'].iloc[i] > 122:
            data.iloc[i, data.columns.get_loc('AMBIENT')] = data['AMBIENT'].iloc[i-1]
            
    data['AMBIENT_F'] = data['AMBIENT']
    data['AMBIENT_C'] = (data['AMBIENT'] - 32)/9*5

    data['AMBIENT_C_diff'] = abs(data['AMBIENT_C'] - data['BETTERY_TEMPERATURA'])
    data['AMBIENT_F_diff'] = abs(data['AMBIENT_F'] - data['BETTERY_TEMPERATURA'])
    ambient_fix = []

    for i in range(len(data)):
        if data.iloc[i]['AMBIENT_C_diff'] <= data.iloc[i]['AMBIENT_F_diff']:
            ambient_fix.append(data.iloc[i]['AMBIENT_C'])
        else:
            ambient_fix.append(data.iloc[i]['AMBIENT_F'])

    data.insert(loc=data.columns.get_loc('AMBIENT')+1, column='AMBIENT_FIX', value=ambient_fix)
    data = data.drop(['AMBIENT_C','AMBIENT_F','AMBIENT_C_diff','AMBIENT_F_diff'], axis=1)
    
    return data

# 途中停車などを考慮し、消費エネルギーを計算する

In [3]:
def calculate_energy(data):
    
    data['SECONDS_DIFF'] = data['SECONDS_DIFF'].shift(1)
    data.loc[0,'SECONDS_DIFF'] = 43200

    DROP_TIME_trip = [x if x>600 else 0 for x in data['SECONDS_DIFF']]
    DIFF_TIME_real = [0 if x>600 else x for x in data['SECONDS_DIFF']]

    data.insert(loc=len(data.columns), column='DROP_TIME', value=DROP_TIME_trip)
    data.insert(loc=len(data.columns), column='DIFF_TIME_real', value=DIFF_TIME_real)
    
    data.JST = pd.to_datetime(data.JST)
    JST_10min_list = []
    
    trip_time = []
    trip_start_time = data.iloc[0]['JST']
    
    sub_id = 1
    sub_id_list = []
    sub_id_list.append(sub_id)
    
    for i in range(len(data)):
        a = pd.Timestamp(dt.datetime(data.JST.iloc[i].year,
                             data.JST.iloc[i].month,
                             data.JST.iloc[i].day,
                             data.JST.iloc[i].hour,int(data.JST.iloc[i].minute/10)*10))
        JST_10min_list.append(a)
        
        if data.iloc[i]['DROP_TIME'] != 0:
            trip_start_time = data.iloc[i]['JST']
            trip_time.append(0)
        else:
            time = (pd.to_datetime(data.iloc[i]['JST']) - pd.to_datetime(trip_start_time)).total_seconds()
            trip_time.append(time)
            
        if i>0:
            if data.iloc[i]['TRIP_ID'] != data.iloc[i-1]['TRIP_ID']:
                sub_id = 1
            else:
                if data.iloc[i]['DROP_TIME'] != 0:
                    sub_id += 1
            sub_id_list.append(sub_id)

    data.insert(loc=data.columns.get_loc('JST')+1, column='JST_10min', value=JST_10min_list)
    data.insert(loc=len(data.columns), column='RUN_TIME', value=trip_time)
    data.insert(loc=data.columns.get_loc('TRIP_ID')+1, column='SUB_ID', value=sub_id_list)
    
    
    data['AUX_POWER'] = list(data.AUX_POWER_100W*100)
    data['AC_POWER'] = list(data.AC_POWER_250W*250)
    data['Est_AC_POWER'] = list(data.ESTIMATED_HEATER_POWER_250W*250+data.ESTIMATED_AC_POWER_50W*50)

    data['ALL_Consumption_Wh'] = list(data.EV_POWER*data.DIFF_TIME_real/3600)
    data['RUN_Consumption_Wh'] = list(data.RUN_POWER*data.DIFF_TIME_real/3600)
    data['AUX_Consumption_Wh'] = list(data.AUX_POWER*data.DIFF_TIME_real/3600)
    data['AC_Consumption_Wh'] = list(data.AC_POWER*data.DIFF_TIME_real/3600)
    data['Est_AC_Consumption_Wh'] = list(data.Est_AC_POWER*data.DIFF_TIME_real/3600)
    
    return data    

# Segmentを振り分ける

In [4]:
def fit_segment_Classifier(segment_data_path):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        segment = pd.read_csv(segment_data_path)
        segment = segment.reset_index()
        segment['SEGMENT_ID'] = segment.index
        segment['segment_central_point_LATITUDE'] = (segment.START_LATITUDED+segment.END_LATITUDE)/2
        segment['segment_central_point_LONGITUDE'] = (segment.START_LONGITUDE+segment.END_LONGITUDE)/2
        train_data = segment[['segment_central_point_LATITUDE','segment_central_point_LONGITUDE']]
        train_label = segment.SEGMENT_ID
        Classifier = KNeighborsClassifier(n_neighbors=1)
        Classifier.fit(train_data,train_label)

        return Classifier

In [5]:
def distribute_segment(data):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        Classifier_home_a = fit_segment_Classifier('segment_101.csv')
        Classifier_home_b = fit_segment_Classifier('segment_102.csv')
        Classifier_out = fit_segment_Classifier('segment_201.csv')

        route = 0
        max_lat = max(list(data.LATITUDE))
        if (max_lat > 35.48) and (data.iloc[0]['TRIP_DIRECTION'] == 'homeward'):
            route = 101
        elif (max_lat < 35.48) and (data.iloc[0]['TRIP_DIRECTION'] == 'homeward'):
            route = 102
        else:
            route = 201

        prediction_data = data.rename(columns={'LATITUDE': 'segment_central_point_LATITUDE', 
                                                      'LONGITUDE': 'segment_central_point_LONGITUDE'})


        if route == 101:
            prediction_data['SEGMENT_ID'] = Classifier_home_a.predict(prediction_data[['segment_central_point_LATITUDE','segment_central_point_LONGITUDE']])
        elif route == 102:
            prediction_data['SEGMENT_ID'] = Classifier_home_b.predict(prediction_data[['segment_central_point_LATITUDE','segment_central_point_LONGITUDE']])
        elif route == 201:
            prediction_data['SEGMENT_ID'] = Classifier_out.predict(prediction_data[['segment_central_point_LATITUDE','segment_central_point_LONGITUDE']])
        else:
            print('set segment ID error!!!')

        return prediction_data['SEGMENT_ID']

# CANデータと気象データの結合

In [6]:
def join_weather_data(weather,leafspy):
    
    weather.DATETIME =  pd.to_datetime(weather.DATETIME)
    for j in range(len(weather)):
        b = pd.Timestamp(weather.DATETIME.iloc[j])
        if (b.hour == 23)and(b.minute == 59):
            b = b + dt.timedelta(minutes = 1)
            weather.iloc[j,weather.columns.get_loc('DATETIME')] = b
            
    absolute_humidity = []
    temperature = np.array(weather['TEMPERATURE'])
    humidity = np.array(weather['HUMIDITY'])
    for i in range(len(temperature)):
        pp = 6.11*10**(7.5*temperature[i]/(temperature[i]+237.3))
        hh = 217*pp/(temperature[i]+273.15)
        hhh = humidity[i]/100*hh
        absolute_humidity.append(hhh)

#     weather = weather.drop(['HUMIDITY'], axis=1)
#     weather.insert(loc=len(weather.columns), column='HUMIDITY', value=absolute_humidity)
    weather['HUMIDITY'] = absolute_humidity

    dataset = pd.merge(leafspy,weather,how='left',left_on='JST_10min',right_on='DATETIME')
    
    return dataset

In [7]:
def join_illuminance_data(illuminance,leafspy):
    
    illuminance = illuminance.drop(['DRIVER_ID','CAR_ID','SENSOR_ID'], axis=1)
    illuminance.DATETIME =  pd.to_datetime(illuminance.DATETIME)

    ill_10min_list = []
    for i in range(len(illuminance)):
        a = pd.Timestamp(dt.datetime(illuminance.DATETIME.iloc[i].year,
                             illuminance.DATETIME.iloc[i].month,
                             illuminance.DATETIME.iloc[i].day,
                             illuminance.DATETIME.iloc[i].hour,int(illuminance.DATETIME.iloc[i].minute/10)*10))
        ill_10min_list.append(a)
    illuminance['ILL_10min'] = ill_10min_list
    
    illuminance_10min = illuminance.groupby('ILL_10min').mean(numeric_only=True)
    illuminance_10min = illuminance_10min.reset_index()
#     illuminance_10min['ILLUMINANCE_Log'] = np.log10((illuminance_10min.ILLUMINANCE+0.1)/2).astype(int)
    
    dataset = pd.merge(leafspy,illuminance_10min,how='left',left_on='JST_10min',right_on='ILL_10min')
    dataset = dataset.drop(['ILL_10min'], axis=1)
        
    return dataset

In [8]:
def join_precipitation(leafspy, precipitation):
    
    precipitation = precipitation[['JST','天気.1','日積算 降水量(mm)']]
    precipitation = precipitation.rename(columns={'天気.1':'WEATHER','日積算 降水量(mm)': 'PRECIPITATION'})
    
    leafspy.START_TIME = pd.to_datetime(leafspy.START_TIME)
    precipitation.JST = pd.to_datetime(precipitation.JST)
    leafspy['DATE']=leafspy['START_TIME'].dt.date
    leafspy['Yesterday']=leafspy['DATE']-dt.timedelta(days=1)
    precipitation['DATE']=precipitation['JST'].dt.date
    
    dataset = pd.merge(leafspy,precipitation,how='left',left_on='DATE',right_on='DATE')
    dataset = dataset.drop(['DATE','JST'], axis=1)
    dataset = pd.merge(dataset,precipitation,how='left',left_on='Yesterday',right_on='DATE')
    dataset = dataset.drop(['DATE','JST','Yesterday'], axis=1)
    dataset = dataset.rename(columns={'PRECIPITATION_x': 'PRECIPITATION','PRECIPITATION_y':'PRECIPITATION_yesterday'})
    dataset = dataset.rename(columns={'WEATHER_x': 'WEATHER','WEATHER_y':'WEATHER_yesterday'})
        
    precipitation1 = []
    precipitation2 = []
    for i in range(len(dataset)):
        precipitation1.append(float(dataset.iloc[i]['PRECIPITATION'][:-2]))
        precipitation2.append(float(dataset.iloc[i]['PRECIPITATION_yesterday'][:-2]))

    dataset = dataset.drop(['PRECIPITATION','PRECIPITATION_yesterday'], axis=1)
    dataset['PRECIPITATION'] = precipitation1
    dataset['PRECIPITATION_yesterday'] = precipitation2
    
    return dataset

In [9]:
def join_precipitation_2(leafspy, precipitation_a, precipitation_b):
    
    precipitation1 = precipitation_a[['DATETIME','PRECIPITATION']].copy()
    precipitation1['PRECIPITATION'] = pd.to_numeric(precipitation1['PRECIPITATION'], errors='coerce').fillna(0)
    
    precipitation2 = precipitation_b[['DATETIME','PRECIPITATION']].copy()
    precipitation2['PRECIPITATION'] = pd.to_numeric(precipitation2['PRECIPITATION'], errors='coerce').fillna(0)
    
    leafspy.START_TIME = pd.to_datetime(leafspy.START_TIME)
    precipitation1.DATETIME = pd.to_datetime(precipitation1.DATETIME)
    precipitation2.DATETIME = pd.to_datetime(precipitation2.DATETIME)
    
    dataset = pd.merge(leafspy,precipitation1,how='left',left_on='START_TIME',right_on='DATETIME')
    dataset = dataset.drop(['DATETIME'], axis=1)
    dataset = pd.merge(dataset,precipitation1,how='left',left_on='END_TIME',right_on='DATETIME')
    dataset = dataset.drop(['DATETIME'], axis=1)
    dataset = dataset.rename(columns={'PRECIPITATION_x': 'PRECIPITATION_START_yokohama','PRECIPITATION_y':'PRECIPITATION_END_yokohama'})
    
    dataset = pd.merge(dataset,precipitation2,how='left',left_on='START_TIME',right_on='DATETIME')
    dataset = dataset.drop(['DATETIME'], axis=1)
    dataset = pd.merge(dataset,precipitation2,how='left',left_on='END_TIME',right_on='DATETIME')
    dataset = dataset.drop(['DATETIME'], axis=1)
    dataset = dataset.rename(columns={'PRECIPITATION_x': 'PRECIPITATION_START_ebina','PRECIPITATION_y':'PRECIPITATION_END_ebina'})
    
    return dataset

In [10]:
def set_weather(dataset, tennki):
    tennki = tennki[['JST','天気.1']]
    tennki = tennki.rename(columns={'天気.1':'WEATHER'})
    tennki['JST'] = pd.to_datetime(tennki['JST'])
    tennki['DATE'] = tennki['JST'].dt.date
    
    dataset.START_TIME = pd.to_datetime(dataset.START_TIME)
    dataset['DATE'] = dataset['START_TIME'].dt.date
    dataset = pd.merge(dataset,tennki,how='left',left_on='DATE',right_on='DATE')
    
    dataset = dataset.drop(['DATE','JST'], axis=1)
    
    return dataset

# Segmentごとにデータを集約する

In [11]:
def aggregated_data(data):
    
    avg_data = data.groupby(['TRIP_ID','SUB_ID','SEGMENT_ID'], as_index=False).mean(numeric_only=True)
    sum_data = data.groupby(['TRIP_ID','SUB_ID','SEGMENT_ID'], as_index=False).sum(numeric_only=True)
    max_data = data.groupby(['TRIP_ID','SUB_ID','SEGMENT_ID'], as_index=False).max(numeric_only=True)
    first_data = data.groupby(['TRIP_ID','SUB_ID','SEGMENT_ID'], as_index=False).first()
    last_data = data.groupby(['TRIP_ID','SUB_ID','SEGMENT_ID'], as_index=False).last()
    # most_data = data.groupby(['TRIP_ID','SUB_ID','SEGMENT_ID']).agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan).reset_index()
    
    segment_dataset = pd.DataFrame()

    segment_dataset.insert(loc=len(segment_dataset.columns), column='TRIP_ID', value=first_data['TRIP_ID'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='SUB_ID', value=first_data['SUB_ID'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='SEGMENT_ID', value=first_data['SEGMENT_ID'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='START_TIME', value=first_data['JST_10min'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='END_TIME', value=last_data['JST_10min'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='START_LATITUDE', value=first_data['LATITUDE'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='START_LONGITUDE', value=first_data['LONGITUDE'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='LAST_LATITUDE', value=last_data['LATITUDE'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='LAST_LONGITUDE', value=last_data['LONGITUDE'].tolist())    
    segment_dataset.insert(loc=len(segment_dataset.columns), column='START_ALTITUDE', value=first_data['ALTITUDE'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='LAST_ALTITUDE', value=last_data['ALTITUDE'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='AVG_ALTITUDE', value=avg_data['ALTITUDE'].tolist())    
    segment_dataset.insert(loc=len(segment_dataset.columns), column='START_SOC', value=first_data['SOC'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='END_SOC', value=last_data['SOC'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='START_GIDS', value=first_data['GIDS'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='END_GIDS', value=last_data['GIDS'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='SOH', value=first_data['SOH'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='AVG_SPEED', value=avg_data['SPEED'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='RUN_TIME', value=last_data['RUN_TIME'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='TRIP_TIME', value=sum_data['DIFF_TIME_real'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='DROP_TIME', value=first_data['DROP_TIME'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='BETTERY_TEMPERATURA', value=avg_data['BETTERY_TEMPERATURA'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='AMBIENT_TEMPERATURA', value=avg_data['AMBIENT_FIX'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='EV_POWER', value=avg_data['EV_POWER'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='RUN_POWER', value=avg_data['RUN_POWER'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='AUX_POWER', value=avg_data['AUX_POWER'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='AC_POWER', value=avg_data['AC_POWER'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='Est_AC_POWER', value=avg_data['Est_AC_POWER'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='ALL_Consumption_Wh', value=sum_data['ALL_Consumption_Wh'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='RUN_Consumption_Wh', value=sum_data['RUN_Consumption_Wh'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='AUX_Consumption_Wh', value=sum_data['AUX_Consumption_Wh'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='AC_Consumption_Wh', value=sum_data['AC_Consumption_Wh'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='EST_AC_Consumption_Wh', value=sum_data['Est_AC_Consumption_Wh'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='TEMPERATURE_yokohama', value=avg_data['TEMPERATURE_yokohama'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='TEMPERATURE_ebina', value=avg_data['TEMPERATURE_ebina'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='HUMIDITY', value=avg_data['HUMIDITY_yokohama'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='WIND_SPEED_yokohama', value=avg_data['WIND_SPEED_yokohama'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='WIND_SPEED_ebina', value=avg_data['WIND_SPEED_ebina'].tolist())
    segment_dataset.insert(loc=len(segment_dataset.columns), column='ILLUMINANCE', value=avg_data['ILLUMINANCE'].tolist())
    
    return segment_dataset

# ここから実行

In [12]:
leafspy = FIX_AMBIENT(leafspy)

dataset = calculate_energy(leafspy)

segment_list = dataset.groupby('TRIP_ID',as_index=False).apply(distribute_segment).reset_index()
dataset['SEGMENT_ID'] = segment_list['SEGMENT_ID']

dataset2 = join_weather_data(weather_yokohama,dataset)
dataset2 = dataset2.drop(['DATETIME','LAST_10MIN_DATETIME','PLACE'], axis=1)
dataset2 = dataset2.rename({'TEMPERATURE':'TEMPERATURE_yokohama','HUMIDITY':'HUMIDITY_yokohama','WIND_SPEED':'WIND_SPEED_yokohama','WIND_DIRECTION':'WIND_DIRECTION_yokohama','PRECIPITATION':'PRECIPITATION_yokohama'},axis='columns')
dataset3 = join_weather_data(weather_ebina,dataset2)
dataset3 = dataset3.drop(['DATETIME','LAST_10MIN_DATETIME','PLACE'], axis=1)
dataset3 = dataset3.rename({'TEMPERATURE':'TEMPERATURE_ebina','HUMIDITY':'HUMIDITY_ebina','WIND_SPEED':'WIND_SPEED_ebina','WIND_DIRECTION':'WIND_DIRECTION_ebina','PRECIPITATION':'PRECIPITATION_ebina'},axis='columns')

dataset4 = join_illuminance_data(illuminance,dataset3)

dataset5 = aggregated_data(dataset4)
dataset6 = join_precipitation_2(dataset5, precipitation_yokohama, precipitation_ebina)
dataset7 = set_weather(dataset6, tennki)

In [13]:
dataset_preparation = dataset7.copy()

dataset_preparation['TEMPERATURE_yokohama'] = dataset_preparation['TEMPERATURE_yokohama'].fillna(dataset_preparation['TEMPERATURE_ebina'])
dataset_preparation['TEMPERATURE_ebina'] = dataset_preparation['TEMPERATURE_ebina'].fillna(dataset_preparation['TEMPERATURE_yokohama'])
dataset_preparation['WIND_SPEED_yokohama'] = dataset_preparation['WIND_SPEED_yokohama'].fillna(dataset_preparation['WIND_SPEED_ebina'])
dataset_preparation['WIND_SPEED_ebina'] = dataset_preparation['WIND_SPEED_ebina'].fillna(dataset_preparation['WIND_SPEED_yokohama'])
dataset_preparation['PRECIPITATION_START_yokohama'] = dataset_preparation['PRECIPITATION_START_yokohama'].fillna(dataset_preparation['PRECIPITATION_START_ebina'])
dataset_preparation['PRECIPITATION_START_ebina'] = dataset_preparation['PRECIPITATION_START_ebina'].fillna(dataset_preparation['PRECIPITATION_START_yokohama'])
dataset_preparation['PRECIPITATION_END_yokohama'] = dataset_preparation['PRECIPITATION_END_yokohama'].fillna(dataset_preparation['PRECIPITATION_END_ebina'])
dataset_preparation['PRECIPITATION_END_ebina'] = dataset_preparation['PRECIPITATION_END_ebina'].fillna(dataset_preparation['PRECIPITATION_END_yokohama'])

In [14]:
print('欠損値削除前のトリップ数：',len(dataset_preparation['TRIP_ID'].unique()))
values_to_remove = dataset_preparation[dataset_preparation.isna().any(axis=1)]['TRIP_ID'].unique()
dataset_final = dataset_preparation[~dataset_preparation['TRIP_ID'].isin(values_to_remove)]
print('欠損値削除後のトリップ数：',len(dataset_final['TRIP_ID'].unique()))

dataset_final = dataset_final.copy()
dataset_final.loc[:, 'ILLUMINANCE_Log'] = np.log10((dataset_final['ILLUMINANCE'] + 0.1) / 2).astype(int)

欠損値削除前のトリップ数： 793
欠損値削除後のトリップ数： 697


In [15]:
temperature_trip_avg = dataset_final.groupby('TRIP_ID').mean(numeric_only=True)[['TEMPERATURE_yokohama','TEMPERATURE_ebina']]
temperature_trip_avg['TRIP_TEMPERATURE'] = (temperature_trip_avg.TEMPERATURE_yokohama + temperature_trip_avg.TEMPERATURE_ebina)/2
temperature_trip_avg = temperature_trip_avg['TRIP_TEMPERATURE'].to_frame()
dataset_final = pd.merge(dataset_final,temperature_trip_avg,how='left',left_on='TRIP_ID',right_on='TRIP_ID')
dataset_final.loc[dataset_final['TEMPERATURE_yokohama'] == -1, 'TEMPERATURE_yokohama'] = dataset_final['TEMPERATURE_ebina']
dataset_final.loc[dataset_final['TEMPERATURE_ebina'] == -1, 'TEMPERATURE_ebina'] = dataset_final['TEMPERATURE_yokohama']

In [16]:
dataset_final.to_csv('dataset.csv')
dataset_final.tail()

,TRIP_ID,SUB_ID,SEGMENT_ID,START_TIME,END_TIME,START_LATITUDE,START_LONGITUDE,LAST_LATITUDE,LAST_LONGITUDE,START_ALTITUDE,LAST_ALTITUDE,AVG_ALTITUDE,START_SOC,END_SOC,START_GIDS,END_GIDS,SOH,AVG_SPEED,RUN_TIME,TRIP_TIME,DROP_TIME,BETTERY_TEMPERATURA,AMBIENT_TEMPERATURA,EV_POWER,RUN_POWER,AUX_POWER,AC_POWER,Est_AC_POWER,ALL_Consumption_Wh,RUN_Consumption_Wh,AUX_Consumption_Wh,AC_Consumption_Wh,EST_AC_Consumption_Wh,TEMPERATURE_yokohama,TEMPERATURE_ebina,HUMIDITY,WIND_SPEED_yokohama,WIND_SPEED_ebina,ILLUMINANCE,PRECIPITATION_START_yokohama,PRECIPITATION_END_yokohama,PRECIPITATION_START_ebina,PRECIPITATION_END_ebina,WEATHER,ILLUMINANCE_Log,TRIP_TEMPERATURE
17464,4560,1,22,2023-08-30 10:50:00,2023-08-30 10:50:00,35.449129,139.567853,35.452739,139.576851,57.7,64.2,59.858333,728318,726240,522,521,91.97,70.764000,3817.0,51.0,0.0,32.358333,32.833333,8988.018412,8038.018413,200.000000,750.000000,666.666667,123.898511,110.440178,2.833333,10.625000,9.444444,31.9,31.799999,20.857681,5.20,3.5,38488.187489,0.0,0.0,0.0,0.0,晴のち曇,4,32.000791
17465,4560,1,23,2023-08-30 10:50:00,2023-08-30 11:00:00,35.452996,139.577632,35.458544,139.585216,60.5,34.8,46.375000,726224,726472,521,521,91.97,70.881001,3868.0,51.0,0.0,32.366667,33.000000,-1109.831677,-2059.831677,200.000000,750.000000,716.666667,-15.617443,-29.075777,2.833333,10.625000,10.166667,31.9,31.974999,21.109992,5.15,3.4,37195.114358,0.0,0.0,0.0,0.0,晴のち曇,4,32.000791
17466,4560,1,24,2023-08-30 11:00:00,2023-08-30 11:00:00,35.459174,139.585967,35.465691,139.591233,32.4,24.4,24.725000,726530,725422,521,520,91.97,69.768000,3918.0,50.0,0.0,32.386111,33.291667,4570.528872,3620.528872,200.000000,750.000000,720.833333,62.262229,49.067784,2.777778,10.416667,10.000000,31.9,32.500000,21.866924,5.00,3.1,33315.894966,0.0,0.0,0.0,0.0,晴のち曇,4,32.000791
17467,4560,1,25,2023-08-30 11:00:00,2023-08-30 11:00:00,35.466307,139.591511,35.472796,139.595000,25.8,49.0,42.200000,725100,722048,520,518,91.97,27.947172,4047.0,129.0,0.0,32.440230,33.500000,4200.582567,3250.582567,200.000000,750.000000,743.103448,132.393101,98.351435,7.166667,26.875000,26.527778,31.9,32.500000,21.866924,5.00,3.1,33315.894966,0.0,0.0,0.0,0.0,晴のち曇,4,32.000791
17468,4560,1,26,2023-08-30 11:00:00,2023-08-30 11:00:00,35.472929,139.594838,35.472286,139.586766,49.3,50.9,48.887097,721812,718416,518,515,91.97,15.419032,4310.0,263.0,0.0,32.493011,33.233871,2664.847759,1729.363888,217.741935,717.741935,740.322581,192.796632,124.282743,15.944444,52.569444,54.180556,31.9,32.500000,21.866924,5.00,3.1,33315.894966,0.0,0.0,0.0,0.0,晴のち曇,4,32.000791
